In [ ]:
from __future__ import division
import sys
sys.path.insert(0, '../../MyPackages/')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2, os, random, time
from tqdm import tqdm

## set global parameters: backend and image_dim_ordering for keras, and the gpu configuration
import os
os.environ['KERAS_BACKEND'] = 'tensorflow' # or 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from keras import backend as K
import tensorflow as tf

K.set_image_dim_ordering('tf')
## for tensorflow sesstion
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement = True
config.log_device_placement = False
sess = tf.Session(config=config)
K.set_session(sess)

print K.image_dim_ordering(), K.backend()

## import keras layers
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from keras.utils.visualize_util import plot
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from inception_v4 import create_inception_v4
from Package_dataset import Load_AVAdataset
from Package_network import loop_batch

#### load the AVA dataset, hdf5 format

In [ ]:
h5f_AVA = Load_AVAdataset(target_size=(224,224))
images_train = h5f_AVA['images_train']
scores_train = h5f_AVA['scores_train']
scores_train = scores_train[:,0]
sample_weights = h5f_AVA['sample_weights_train'][:]
sample_weights[sample_weights>50] = 50

images_test_even = h5f_AVA['images_test_even']
scores_test_even = h5f_AVA['scores_test_even']
scores_test_even = scores_test_even[:,0]
images_test_uneven = h5f_AVA['images_test_uneven']
scores_test_uneven = h5f_AVA['scores_test_uneven']
scores_test_uneven = scores_test_uneven[:,0]

print images_train.shape, scores_train.shape, sample_weights.shape
print images_test_even.shape, scores_test_even.shape
print images_test_uneven.shape, scores_test_uneven.shape


#### Pre-process functions

In [ ]:
def preprocess_input(x, dim_ordering='default'): # pre-process for Resnet and VGG
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 4D.
        dim_ordering: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}

    x = x.astype(np.float32,copy=False)
    if dim_ordering == 'th':
        # 'RGB'->'BGR'
#         x = x[:, ::-1, :, :]
        # Zero-center by mean pixel
        x[:, 0, :, :] -= 103.939 # blue
        x[:, 1, :, :] -= 116.779 # green
        x[:, 2, :, :] -= 123.68  # red
    else:
        x = np.transpose(x,(0,2,3,1))
        # 'RGB'->'BGR'
#         x = x[:, :, :, ::-1]
        # Zero-center by mean pixel
        x[:, :, :, 0] -= 103.939 # blue
        x[:, :, :, 1] -= 116.779 # green
        x[:, :, :, 2] -= 123.68  # red
    return x

In [ ]:
def preprocess_input(x,dim_ordering='default'): # pre-process for Xception and Inceptioin
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}
    
    x = x.astype(np.float32,copy=False)
    if dim_ordering == 'tf':
        x = np.transpose(x,(0,2,3,1))

    x /= 255.
    x -= 0.5
    x *= 2.
    return x

#### Define and build the network

In [ ]:
with K.tf.device('/:gpu0'):
    input_tensor = Input(shape=(224, 224, 3))
    base_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(1,init='glorot_uniform',activation='sigmoid')(x) # in the range of 0 to 1
    model = Model(input=input_tensor, output=x)
#     model.load_weights('/data/bjin/MyAesthetics/model_weights/AVA_Xception_score_sigmoid.hdf5')
    myadam = Adam(lr=0.0001)
    model.compile(optimizer=myadam, loss='mse')


In [ ]:
images_test_even = preprocess_input(images_test_even[:])
print images_test_even.shape

In [ ]:
st_time = time.time()
scores_all = model.predict(images_test_even,batch_size=128)
print time.time()-st_time
print np.mean(np.square(np.squeeze(scores_all)*10 - scores_test_even))
print np.corrcoef(np.squeeze(scores_all)*10, scores_test_even)

#### Train the Xception network for aesthetic scores prediction

In [ ]:
## define Xception network
with K.tf.device('/:gpu0'):
    input_tensor = Input(shape=(224, 224, 3))
    base_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(1,init='glorot_uniform',activation='sigmoid')(x) # in the range of 0 to 1
    model = Model(input=input_tensor, output=x)
#     model.load_weights('/data/bjin/MyAesthetics/model_weights/AVA_Xception_score_sigmoid.hdf5')
    myadam = Adam(lr=0.0001)
    model.compile(optimizer=myadam, loss='mse')

In [ ]:
ff = open('/data/bjin/MyAesthetics/logs/AVA_Xception_score_sigmoid.txt','w+')

# first finetune the top layer
print >>ff, 'finetune the top layer'
for l in model.layers[:-1]:
    l.trainable = False
model.compile(optimizer=myadam, loss='mse')

checkpointer = ModelCheckpoint(filepath="/data/bjin/MyAesthetics/model_weights/AVA_Xception_score_sigmoid.hdf5", 
                                verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.000001)

history = model.fit_generator(loop_batch(images_train,scores_train/10,sample_weights,batch_size=128,pre_f=preprocess_input,shuffle=True),
        samples_per_epoch=len(scores_train), nb_epoch=5, callbacks=[checkpointer, reduce_lr], verbose=1,
        validation_data=loop_batch(images_test_even,scores_test_even/10,batch_size=128,pre_f=preprocess_input),
        nb_val_samples=len(scores_test_even), max_q_size=5, nb_worker=1, pickle_safe=False, initial_epoch=0)

print >>ff, history.history

# then finetune all the layers
print >>ff, 'finetune all the layers'
for l in model.layers[:-1]:
    l.trainable = True
model.load_weights('/data/bjin/MyAesthetics/model_weights/AVA_Xception_score_sigmoid.hdf5')
myadam = Adam(lr=0.00005)
model.compile(optimizer=myadam, loss='mse')

checkpointer = ModelCheckpoint(filepath="/data/bjin/MyAesthetics/model_weights/AVA_Xception_score_sigmoid.hdf5", 
                                verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.000001)

history = model.fit_generator(loop_batch(images_train,scores_train/10,sample_weights,batch_size=32,pre_f=preprocess_input,shuffle=True),
        samples_per_epoch=len(scores_train), nb_epoch=15, callbacks=[checkpointer, reduce_lr], verbose=1,
        validation_data=loop_batch(images_test_even,scores_test_even/10,batch_size=32,pre_f=preprocess_input),
        nb_val_samples=len(scores_test_even), max_q_size=5, nb_worker=1, pickle_safe=False, initial_epoch=0)

print >>ff, history.history

#### Train the Resnet network for aesthetic scores prediction

In [ ]:
## define Resnet network
with K.tf.device('/gpu:0'):
    input_tensor = Input(shape=(224, 224, 3))
    base_model = ResNet50(input_tensor=input_tensor, include_top=False, weights='imagenet')
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1,init='glorot_uniform',activation='sigmoid')(x) # in the range of 0 to 1
    model = Model(input=input_tensor, output=x)
    myadam = Adam(lr=0.0001)

In [ ]:
ff = open('/data/bjin/MyAesthetics/logs/AVA_Resnet_score_sigmoid.txt','w+')

# first finetune the top layer
print >>ff, 'finetune the top layer'
for l in model.layers[:-1]:
    l.trainable = False
model.compile(optimizer=myadam, loss='mse')

checkpointer = ModelCheckpoint(filepath="/data/bjin/MyAesthetics/model_weights/AVA_Resnet50_score_sigmoid.hdf5", 
                                verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.000001)
mytensorboard = TensorBoard(log_dir='/data/bjin/MyAesthetics/logs/Resnet', histogram_freq=5, write_graph=False)

history = model.fit_generator(loop_batch(images_train,scores_train/10,sample_weights,batch_size=64,pre_f=preprocess_input),
        samples_per_epoch=len(scores_train), nb_epoch=5, callbacks=[checkpointer, reduce_lr, mytensorboard], verbose=1,
        validation_data=loop_batch(images_test_even,scores_test_even/10,batch_size=64,pre_f=preprocess_input),
        nb_val_samples=len(scores_test_even), max_q_size=5, nb_worker=1, pickle_safe=False, initial_epoch=0)

print >>ff, history.history

# then finetune all the layers
print >>ff, 'finetune all the layers'
for l in model.layers[:-1]:
    l.trainable = True
model.load_weights('/data/bjin/MyAesthetics/model_weights/AVA_Resnet50_score_sigmoid.hdf5')
myadam = Adam(lr=0.0001)
model.compile(optimizer=myadam, loss='mse')

checkpointer = ModelCheckpoint(filepath="/data/bjin/MyAesthetics/model_weights/AVA_Resnet50_score_sigmoid.hdf5", 
                                verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.000001)
mytensorboard = TensorBoard(log_dir='/data/bjin/MyAesthetics/logs/Resnet', histogram_freq=5, write_graph=False)

history = model.fit_generator(loop_batch(images_train,scores_train/10,sample_weights,batch_size=64,pre_f=preprocess_input),
        samples_per_epoch=len(scores_train), nb_epoch=15, callbacks=[checkpointer, reduce_lr, mytensorboard], verbose=1,
        validation_data=loop_batch(images_test_even,scores_test_even/10,batch_size=64,pre_f=preprocess_input),
        nb_val_samples=len(scores_test_even), max_q_size=5, nb_worker=1, pickle_safe=False, initial_epoch=0)

print >>ff, history.history


#### Backup codes used for generating figures for statistics

In [ ]:
## generate the statistics
path_AVA = '/data/bjin/AVA/'
metadata = loadmat(path_AVA + 'AVA_metadata.mat')
files = metadata['img_names'][0,:]
GTscores = metadata['scores']
n_files = len(files)
print n_files
idx = np.random.permutation(n_files)
files = files[idx]
GTscores = GTscores[:,idx]

path_results = '/home/bjin/SampleImages/'

files = metadata['img_names'][0,:]
GTscores = metadata['scores']
mystd = np.std(GTscores[1:,:],axis=0)
idx = np.argsort(mystd)
files = files[idx]
GTscores = GTscores[:,idx]

for filename, GTscore in zip(files,GTscores.transpose([1,0])):
#     if GTscore[0] > 7 or GTscore[0] < 5:
#         continue
    
    img = cv2.imread(path_AVA + 'img/' + str(filename[0]))
    imgn = np.copy(img)
    imgn = cv2.resize(imgn,(224,224))
    imgn = np.transpose(imgn,[2,0,1])
    imgn = np.expand_dims(imgn,axis=0)
    score = model.predict(imgn)
    
    fig = plt.figure(figsize=(20, 4))
    ax1 = fig.add_subplot(121)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    ax1.set_title('ori')
    ax1.axis('off')
    
    ax2 = fig.add_subplot(122)
    plt.bar(range(10),GTscore[1:]/np.sum(GTscore[1:]),width=0.9,align='center')
    plt.xticks(range(1,11),('1', '2', '3', '4', '5', '6','7','8','9','10'))
    plt.xlim(.5, 10.5)
    ax2.set_title('hist')
    plt.show()
    print score, GTscore
    
    flag = raw_input('press "s" to save: ')
    if flag == 's':
        extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(path_results + str(filename[0]), bbox_inches=extent)

        extent = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(path_results + 'hist_' + str(filename[0]), bbox_inches=extent.expanded(1.1, 1.2))
    
    plt.clf()

In [ ]:
plot(model_inceptionv4, to_file='model.png')